# Setup

In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
#quiet_library(Nebulosa)
quiet_library(ArchR)

In [ ]:
library(dplyr)
library(data.table)
library(ArchR)
library(plyranges)
library(doParallel)
library(MASS)
library(reshape)
library(ggplot2)
library(ggforce)
library(metR)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(BSgenome.Hsapiens.UCSC.hg38)
library(org.Hs.eg.db)

In [ ]:
addArchRThreads(32)
addArchRGenome("hg38")

In [ ]:
proj <- loadArchRProject(path = '../PedSen_ATAC/')
proj

In [ ]:
getAvailableMatrices(proj)

# Frags & Size Analysis

In [ ]:
median(proj$nFrags)

## Subset Pediatric & Senior Projects

In [ ]:
ped_idx <- which(proj$pediatric_senior == 'Pediatric')
ped_cells <- proj$cellNames[ped_idx]
ped_proj <- proj[ped_cells, ]

sen_idx <- which(proj$pediatric_senior == 'Senior')
sen_cells <- proj$cellNames[sen_idx]
sen_proj <- proj[sen_cells, ]

## Subset by Batch

In [ ]:
batch_ids <- c('B065','B069','B076')

In [ ]:
ped_proj_list <- lapply(batch_ids, function(x){
    idx <- which(ped_proj$batch_id == x)
    proj_cells <- ped_proj$cellNames[idx]
    proj_subset <- ped_proj[proj_cells, ]
    return(proj_subset)
})
names(ped_proj_list) <- batch_ids

In [ ]:
sen_proj_list <- lapply(batch_ids, function(x){
    idx <- which(sen_proj$batch_id == x)
    proj_cells <- sen_proj$cellNames[idx]
    proj_subset <- sen_proj[proj_cells, ]
    return(proj_subset)
})
names(sen_proj_list) <- batch_ids

## Get Frags per cell 

In [ ]:
gating_celltypes <- unique(proj$gating_celltype)
gating_celltypes

In [ ]:
median_frags_celltype_func <- function(celltype,archr_project){
    idx <- which(archr_project$gating_celltype %in% celltype)
    proj_cells <- archr_project$cellNames[idx]
    proj_subset <- archr_project[proj_cells, ]
    return(median(proj_subset$nFrags))}

In [ ]:
ped_b065_celltype_frags <- lapply(gating_celltypes, median_frags_celltype_func, ped_proj_list$B065)
names(ped_b065_celltype_frags) <- gating_celltypes
sen_b065_celltype_frags <- lapply(gating_celltypes, median_frags_celltype_func, sen_proj_list$B065)
names(sen_b065_celltype_frags) <- gating_celltypes

In [ ]:
ped_b069_celltype_frags <- lapply(gating_celltypes, median_frags_celltype_func, ped_proj_list$B069)
names(ped_b069_celltype_frags) <- gating_celltypes
sen_b069_celltype_frags <- lapply(gating_celltypes, median_frags_celltype_func, sen_proj_list$B069)
names(sen_b069_celltype_frags) <- gating_celltypes

In [ ]:
ped_b076_celltype_frags <- lapply(gating_celltypes, median_frags_celltype_func, ped_proj_list$B076)
names(ped_b076_celltype_frags) <- gating_celltypes
sen_b076_celltype_frags <- lapply(gating_celltypes, median_frags_celltype_func, sen_proj_list$B076)
names(sen_b076_celltype_frags) <- gating_celltypes

In [ ]:
frag_df_combined <- rbind(ped_b065_celltype_frags, ped_b069_celltype_frags, ped_b076_celltype_frags,
                          sen_b065_celltype_frags, sen_b069_celltype_frags, sen_b076_celltype_frags)

In [ ]:
frag_df_combined

In [ ]:
write.csv(frag_df_combined, file = '../frag_df_combined.csv')

# DAPs per cell type by Age and by CMV status

Find differntially accessible peaks within each gated cell type by either age group or CMV status. Trying to maximize the number of cells used but only using the minimum overlap between Age & CMV status so that you can compare within cell types. Can not compare between cell types with this strategy. 

## Create metadata slots needed for this analysis

In [ ]:
proj$age_celltype <- paste0(proj$pediatric_senior,"_",proj$gating_celltype)
proj$cmv_celltype <- paste0(proj$CMV,"_",proj$gating_celltype)

## Functions

In [ ]:
celltypes <- unique(proj$gating_celltype)

In [ ]:
cmv_status <- unique(proj$CMV)
cmv_status

In [ ]:
sen_age_marker_test_func <- function(celltype, archr_project){
    
    dap_list = list()
    # Get maximum number of cells to use
    ## Get pediatric number
    idx <- which(archr_project$age_celltype %in% paste0('Pediatric_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    ped_age <- length(proj_subset1$cellNames)
    ## Get senior number
    idx <- which(archr_project$age_celltype %in% paste0('Senior_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    sen_age <- length(proj_subset1$cellNames)
    ## Get pediatric number
    idx <- which(archr_project$cmv_celltype %in% paste0('Positive_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    cmv_pos <- length(proj_subset1$cellNames)
    ## Get senior number
    idx <- which(archr_project$cmv_celltype %in% paste0('Negative_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    cmv_neg <- length(proj_subset1$cellNames)
    ## Use minimum as maxCells
    cell_number <- min(ped_age,sen_age,cmv_pos,cmv_neg)
    print(cell_number)
    
    # Get marker peaks
    markerTest <- getMarkerFeatures(
                      ArchRProj = archr_project, 
                      useMatrix = "PeakMatrix",
                      groupBy = "age_celltype",
                      testMethod = "wilcoxon",
                      bias = c("TSSEnrichment", "log10(nFrags)"),
                      useGroups = paste0('Senior_',celltype),
                      bgdGroups = paste0('Pediatric_',celltype),
                      maxCells = cell_number
                    )
    return(markerTest)
    # Extract Significant peaks
#     name <- paste0('Senior_',celltype)
#     sig_markers_up <- getMarkers(markerTest, cutOff = "FDR <= 0.1 & Log2FC >= 0.5")
#     sig_markers_df <- sig_markers_up[[name]]
    
#     return(sig_markers_df)
}   

In [ ]:
ped_age_marker_test_func <- function(celltype, archr_project){
    
    dap_list = list()
    # Get maximum number of cells to use
    ## Get pediatric number
    idx <- which(archr_project$age_celltype %in% paste0('Pediatric_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    ped_age <- length(proj_subset1$cellNames)
    ## Get senior number
    idx <- which(archr_project$age_celltype %in% paste0('Senior_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    sen_age <- length(proj_subset1$cellNames)
    ## Get pediatric number
    idx <- which(archr_project$cmv_celltype %in% paste0('Positive_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    cmv_pos <- length(proj_subset1$cellNames)
    ## Get senior number
    idx <- which(archr_project$cmv_celltype %in% paste0('Negative_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    cmv_neg <- length(proj_subset1$cellNames)
    ## Use minimum as maxCells
    cell_number <- min(ped_age,sen_age,cmv_pos,cmv_neg)
    print(cell_number)
    
    # Get marker peaks
    markerTest <- getMarkerFeatures(
                      ArchRProj = archr_project, 
                      useMatrix = "PeakMatrix",
                      groupBy = "age_celltype",
                      testMethod = "wilcoxon",
                      bias = c("TSSEnrichment", "log10(nFrags)"),
                      useGroups = paste0('Pediatric_',celltype),
                      bgdGroups = paste0('Senior_',celltype),
                      maxCells = cell_number
                    )
    return(markerTest)
    # Extract Significant peaks
#     name <- paste0('Pediatric_',celltype)
#     sig_markers_up <- getMarkers(markerTest, cutOff = "FDR <= 0.1 & Log2FC >= 0.5")
#     sig_markers_df <- sig_markers_up[[name]]
    
#     return(sig_markers_df)
}

In [ ]:
cmv_pos_marker_test_func <- function(celltype, archr_project){
    
    dap_list = list()
    # Get maximum number of cells to use
    ## Get pediatric number
    idx <- which(archr_project$age_celltype %in% paste0('Pediatric_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    ped_age <- length(proj_subset1$cellNames)
    ## Get senior number
    idx <- which(archr_project$age_celltype %in% paste0('Senior_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    sen_age <- length(proj_subset1$cellNames)
    ## Get pediatric number
    idx <- which(archr_project$cmv_celltype %in% paste0('Positive_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    cmv_pos <- length(proj_subset1$cellNames)
    ## Get senior number
    idx <- which(archr_project$cmv_celltype %in% paste0('Negative_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    cmv_neg <- length(proj_subset1$cellNames)
    ## Use minimum as maxCells
    cell_number <- min(ped_age,sen_age,cmv_pos,cmv_neg)
    print(cell_number)
    
    # Get marker peaks
    markerTest <- getMarkerFeatures(
                      ArchRProj = archr_project, 
                      useMatrix = "PeakMatrix",
                      groupBy = "cmv_celltype",
                      testMethod = "wilcoxon",
                      bias = c("TSSEnrichment", "log10(nFrags)"),
                      useGroups = paste0('Positive_',celltype),
                      bgdGroups = paste0('Negative_',celltype),
                      maxCells = cell_number
                    )
    return(markerTest)
    # Extract Significant peaks
#     name <- paste0('Positive_',celltype)
#     sig_markers_up <- getMarkers(markerTest, cutOff = "FDR <= 0.1 & Log2FC >= 0.5")
#     sig_markers_df <- sig_markers_up[[name]]
    
#     return(sig_markers_df)
}

In [ ]:
cmv_neg_marker_test_func <- function(celltype, archr_project){
    
    dap_list = list()
    # Get maximum number of cells to use
    ## Get pediatric number
    idx <- which(archr_project$age_celltype %in% paste0('Pediatric_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    ped_age <- length(proj_subset1$cellNames)
    ## Get senior number
    idx <- which(archr_project$age_celltype %in% paste0('Senior_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    sen_age <- length(proj_subset1$cellNames)
    ## Get pediatric number
    idx <- which(archr_project$cmv_celltype %in% paste0('Positive_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    cmv_pos <- length(proj_subset1$cellNames)
    ## Get senior number
    idx <- which(archr_project$cmv_celltype %in% paste0('Negative_',celltype))
    proj_cells <- archr_project$cellNames[idx]
    proj_subset1 <- archr_project[proj_cells,]
    cmv_neg <- length(proj_subset1$cellNames)
    ## Use minimum as maxCells
    cell_number <- min(ped_age,sen_age,cmv_pos,cmv_neg)
    print(cell_number)
    
    # Get marker peaks
    markerTest <- getMarkerFeatures(
                      ArchRProj = archr_project, 
                      useMatrix = "PeakMatrix",
                      groupBy = "cmv_celltype",
                      testMethod = "wilcoxon",
                      bias = c("TSSEnrichment", "log10(nFrags)"),
                      useGroups = paste0('Negative_',celltype),
                      bgdGroups = paste0('Positive_',celltype),
                      maxCells = cell_number
                    )
    return(markerTest)
    # Extract Significant peaks
#     name <- paste0('Negative_',celltype)
#     sig_markers_up <- getMarkers(markerTest, cutOff = "FDR <= 0.1 & Log2FC >= 0.5")
#     sig_markers_df <- sig_markers_up[[name]]
    
#     return(sig_markers_df)
}

In [ ]:
fastAnnoPeaks <- function(peaks = NULL, 
                          genomeAnnotation = BSgenome.Hsapiens.UCSC.hg38, 
                          TxDb = TxDb.Hsapiens.UCSC.hg38.knownGene, 
                          Org = org.Hs.eg.db, 
                          promoterRegion = c(2000, 100)){
  
  BSgenome = BSgenome.Hsapiens.UCSC.hg38
  geneAnnotation = createGeneAnnotation(TxDb = TxDb, OrgDb = Org)
	
  #Validate
  peaks <- validGRanges(peaks)
  peakSummits <- resize(peaks,1,"center")
  geneAnnotation$genes <- validGRanges(geneAnnotation$genes)
  geneAnnotation$exons <- validGRanges(geneAnnotation$exons)
  geneAnnotation$TSS <- validGRanges(geneAnnotation$TSS)
  BSgenome <- validBSgenome(BSgenome)
  
  #First Lets Get Distance to Nearest Gene Start
  
  distPeaks <- distanceToNearest(peakSummits, resize(geneAnnotation$genes, 1, "start"), ignore.strand = TRUE)
  mcols(peaks)$distToGeneStart <- mcols(distPeaks)$distance
  mcols(peaks)$nearestGene <- mcols(geneAnnotation$genes)$symbol[subjectHits(distPeaks)]
  
  promoters <- extendGR(resize(geneAnnotation$genes, 1, "start"), upstream = promoterRegion[1], downstream = promoterRegion[2])
  op <- overlapsAny(peakSummits, promoters, ignore.strand = TRUE)
  og <- overlapsAny(peakSummits, geneAnnotation$genes, ignore.strand = TRUE)
  oe <- overlapsAny(peakSummits, geneAnnotation$exons, ignore.strand = TRUE)
  type <- rep("Distal", length(peaks))
  type[which(og & oe)] <- "Exonic"
  type[which(og & !oe)] <- "Intronic"
  type[which(op)] <- "Promoter"
  mcols(peaks)$peakType <- type
  
  #First Lets Get Distance to Nearest TSS's
  distTSS <- distanceToNearest(peakSummits, resize(geneAnnotation$TSS, 1, "start"), ignore.strand = TRUE)
  mcols(peaks)$distToTSS <- mcols(distTSS)$distance
  if("symbol" %in% colnames(mcols(geneAnnotation$TSS))){
    mcols(peaks)$nearestTSS <- mcols(geneAnnotation$TSS)$symbol[subjectHits(distPeaks)]
  }else if("tx_name" %in% colnames(mcols(geneAnnotation$TSS))){
    mcols(peaks)$nearestTSS <- mcols(geneAnnotation$TSS)$tx_name[subjectHits(distPeaks)]
  }
  
  #Get NucleoTide Content
  nucFreq <- BSgenome::alphabetFrequency(getSeq(BSgenome, peaks))
  mcols(peaks)$GC <- round(rowSums(nucFreq[,c("G","C")]) / rowSums(nucFreq),4)
  mcols(peaks)$N <- round(nucFreq[,c("N")] / rowSums(nucFreq),4)
  peaks
  
}

#Support function
validGRanges <- function(gr = NULL){
  stopifnot(!is.null(gr))
  if(inherits(gr, "GRanges")){
    return(gr)
  }else{
    stop("Error cannot validate genomic range!")
  }
}

## Obtain DAPs

In [ ]:
sen_age_markers_up <- lapply(celltypes, sen_age_marker_test_func, proj)

In [ ]:
ped_age_markers_up <- lapply(celltypes, ped_age_marker_test_func, proj)

In [ ]:
cmv_pos_markers_up <- lapply(celltypes, cmv_pos_marker_test_func, proj)

In [ ]:
cmv_neg_markers_up <- lapply(celltypes, cmv_neg_marker_test_func, proj)

## Save Marker Test

In [ ]:
saveRDS(sen_age_markers_up, file = '../02_Gating_Subsets/sen_age_markers_up.rds')
saveRDS(ped_age_markers_up, file = '../02_Gating_Subsets/ped_age_markers_up.rds')
saveRDS(cmv_pos_markers_up, file = '../02_Gating_Subsets/cmv_pos_markers_up.rds')
saveRDS(cmv_neg_markers_up, file = '../02_Gating_Subsets/cmv_neg_markers_up.rds')

In [ ]:
sen_age_markers_up <- readRDS(file = '../02_Gating_Subsets/sen_age_markers_up.rds')
ped_age_markers_up <- readRDS(file = '../02_Gating_Subsets/ped_age_markers_up.rds')
cmv_pos_markers_up <- readRDS(file = '../02_Gating_Subsets/cmv_pos_markers_up.rds')
cmv_neg_markers_up <- readRDS(file = '../02_Gating_Subsets/cmv_neg_markers_up.rds')

## Annotate Peaks

Let's look at peaks for pediatric, senior, cmv pos, cmv neg at log2FC 0.5 & save those for plotting in the other notebook

In [ ]:
peak_loc_func <- function(markerTest_output){
    gr <- getMarkers(markerTest_output, cutOff = "FDR <= 0.1 & Log2FC >= 0.5", returnGR = TRUE)
    fa <- fastAnnoPeaks(peaks = gr[[1]])
    df <- as.data.frame(table(summarize(fa, peakType)))
    return(df)
    }

In [ ]:
sen_loc_df <- lapply(sen_age_markers_up,peak_loc_func)

In [ ]:
ped_loc_df <- lapply(ped_age_markers_up,peak_loc_func)

In [ ]:
cmvpos_loc_df <- lapply(cmv_pos_markers_up,peak_loc_func)

In [ ]:
cmvneg_loc_df <- lapply(cmv_neg_markers_up,peak_loc_func)

In [ ]:
names(sen_loc_df) <- celltypes
names(ped_loc_df) <- celltypes
names(cmvpos_loc_df) <- celltypes
names(cmvneg_loc_df) <- celltypes

In [ ]:
saveRDS(sen_loc_df, file = '../02_Gating_Subsets/sen_loc_df.rds')
saveRDS(ped_loc_df, file = '../02_Gating_Subsets/ped_loc_df.rds')
saveRDS(cmvpos_loc_df, file = '../02_Gating_Subsets/cmvpos_loc_df.rds')
saveRDS(cmvneg_loc_df, file = '../02_Gating_Subsets/cmvneg_loc_df.rds')

In [ ]:
sen_perc <- lapply(sen_loc_df, function(x){
    freq_sum <- sum(x$Freq)
    x$perc <- x$Freq / freq_sum
    return(x)
    })
# sen_perc
ped_perc <- lapply(ped_loc_df, function(x){
    freq_sum <- sum(x$Freq)
    x$perc <- x$Freq / freq_sum
    return(x)
    })
# ped_perc

In [ ]:
sen_perc[[1]]

In [ ]:
sen_distal <- lapply(sen_perc, function(x){
                        new_subset <- x[x$Var1 == 'Distal',]
                        return(new_subset$perc)
                        })
sen_exonic <- lapply(sen_perc, function(x){
                        new_subset <- x[x$Var1 == 'Exonic',]
                        return(new_subset$perc)
                        })
sen_intronic <- lapply(sen_perc, function(x){
                        new_subset <- x[x$Var1 == 'Intronic',]
                        return(new_subset$perc)
                        })
sen_promoter <- lapply(sen_perc, function(x){
                        new_subset <- x[x$Var1 == 'Promoter',]
                        return(new_subset$perc)
                        })

In [ ]:
sen_df_perc <- data.frame(distal = unlist(sen_distal),
                          exonic = unlist(sen_exonic),
                          intronic = unlist(sen_intronic),
                          promoter = unlist(sen_promoter))
sen_df_perc$age <- rep('Senior', length(rownames(sen_df_perc)))
sen_df_perc$celltype <- rownames(sen_df_perc)
sen_df_perc

In [ ]:
ped_distal <- lapply(ped_perc, function(x){
                        new_subset <- x[x$Var1 == 'Distal',]
                        return(new_subset$perc)
                        })
ped_exonic <- lapply(ped_perc, function(x){
                        new_subset <- x[x$Var1 == 'Exonic',]
                        return(new_subset$perc)
                        })
ped_intronic <- lapply(ped_perc, function(x){
                        new_subset <- x[x$Var1 == 'Intronic',]
                        return(new_subset$perc)
                        })
ped_promoter <- lapply(ped_perc, function(x){
                        new_subset <- x[x$Var1 == 'Promoter',]
                        return(new_subset$perc)
                        })

In [ ]:
ped_df_perc <- data.frame(distal = unlist(ped_distal),
                          exonic = unlist(ped_exonic),
                          intronic = unlist(ped_intronic),
                          promoter = unlist(ped_promoter))
ped_df_perc$age <- rep('Pediatric', length(rownames(ped_df_perc)))
ped_df_perc$celltype <- rownames(ped_df_perc)
ped_df_perc

In [ ]:
combined_df_perc <- rbind(sen_df_perc, ped_df_perc)

In [ ]:
library(tidyr)

In [ ]:
perc_tall <- gather(data = combined_df_perc, key = comparison, value = perc, `distal`:`promoter`, factor_key = TRUE)
head(perc_tall)

In [ ]:
perc_tall$age_celltype <- paste0(perc_tall$celltype,"_",perc_tall$age)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6)
ggplot(perc_tall, aes(fill=comparison, y=perc, x=age_celltype)) + 
    geom_bar(position="fill", stat="identity") + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

In [ ]:
perc_tall_subset <- subset(perc_tall, age_celltype %in% c('CD8 Naive_Pediatric','CD8 TEMRA_Pediatric','CD8 Naive_Senior','CD8 TEMRA_Senior'))
# perc_tall_subset

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
ggplot(perc_tall_subset, aes(fill=comparison, y=perc, x=age_celltype)) +  
    geom_bar(position="fill", stat="identity") + geom_text(aes(label=round(perc,digits = 3)),position=position_stack(vjust = 0.5)) +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

In [ ]:
peak_loc_func <- function(markerTest_output){
    gr <- getMarkers(markerTest_output, cutOff = "FDR <= 0.1 & Log2FC >= 1", returnGR = TRUE)
    fa <- fastAnnoPeaks(peaks = gr[[1]])
    df <- as.data.frame(table(summarize(fa, peakType)))
    return(df)
    }

In [ ]:
sen_loc_df <- lapply(sen_age_markers_up,peak_loc_func)

In [ ]:
ped_loc_df <- lapply(ped_age_markers_up,peak_loc_func)

In [ ]:
names(sen_loc_df) <- celltypes
names(ped_loc_df) <- celltypes

In [ ]:
head(sen_loc_df)

In [ ]:
sen_perc <- lapply(sen_loc_df, function(x){
    freq_sum <- sum(x$Freq)
    x$perc <- x$Freq / freq_sum
    return(x)
    })
# sen_perc
ped_perc <- lapply(ped_loc_df, function(x){
    freq_sum <- sum(x$Freq)
    x$perc <- x$Freq / freq_sum
    return(x)
    })
# ped_perc

In [ ]:
sen_perc[[1]]

In [ ]:
sen_distal <- lapply(sen_perc, function(x){
                        new_subset <- x[x$Var1 == 'Distal',]
                        return(new_subset$perc)
                        })
sen_exonic <- lapply(sen_perc, function(x){
                        new_subset <- x[x$Var1 == 'Exonic',]
                        return(new_subset$perc)
                        })
sen_intronic <- lapply(sen_perc, function(x){
                        new_subset <- x[x$Var1 == 'Intronic',]
                        return(new_subset$perc)
                        })
sen_promoter <- lapply(sen_perc, function(x){
                        new_subset <- x[x$Var1 == 'Promoter',]
                        return(new_subset$perc)
                        })

In [ ]:
sen_df_perc <- data.frame(distal = unlist(sen_distal),
                          exonic = unlist(sen_exonic),
                          intronic = unlist(sen_intronic),
                          promoter = unlist(sen_promoter))
sen_df_perc$age <- rep('Senior', length(rownames(sen_df_perc)))
sen_df_perc$celltype <- rownames(sen_df_perc)
sen_df_perc

In [ ]:
ped_distal <- lapply(ped_perc, function(x){
                        new_subset <- x[x$Var1 == 'Distal',]
                        return(new_subset$perc)
                        })
ped_exonic <- lapply(ped_perc, function(x){
                        new_subset <- x[x$Var1 == 'Exonic',]
                        return(new_subset$perc)
                        })
ped_intronic <- lapply(ped_perc, function(x){
                        new_subset <- x[x$Var1 == 'Intronic',]
                        return(new_subset$perc)
                        })
ped_promoter <- lapply(ped_perc, function(x){
                        new_subset <- x[x$Var1 == 'Promoter',]
                        return(new_subset$perc)
                        })

In [ ]:
ped_df_perc <- data.frame(distal = unlist(ped_distal),
                          exonic = unlist(ped_exonic),
                          intronic = unlist(ped_intronic),
                          promoter = unlist(ped_promoter))
ped_df_perc$age <- rep('Pediatric', length(rownames(ped_df_perc)))
ped_df_perc$celltype <- rownames(ped_df_perc)
ped_df_perc

In [ ]:
combined_df_perc <- rbind(sen_df_perc, ped_df_perc)

In [ ]:
library(tidyr)

In [ ]:
perc_tall <- gather(data = combined_df_perc, key = comparison, value = perc, `distal`:`promoter`, factor_key = TRUE)
head(perc_tall)

In [ ]:
perc_tall$age_celltype <- paste0(perc_tall$celltype,"_",perc_tall$age)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6)
ggplot(perc_tall, aes(fill=comparison, y=perc, x=age_celltype)) + 
    geom_bar(position="fill", stat="identity") + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

In [ ]:
perc_tall_subset <- subset(perc_tall, age_celltype %in% c('CD8 Naive_Pediatric','CD8 TEMRA_Pediatric','CD8 Naive_Senior','CD8 TEMRA_Senior'))
# perc_tall_subset

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
ggplot(perc_tall_subset, aes(fill=comparison, y=perc, x=age_celltype)) +  
    geom_bar(position="fill", stat="identity") + geom_text(aes(label=round(perc,digits = 3)),position=position_stack(vjust = 0.5)) +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

# Motif Analysis per cell type

Extract motifs from the peak tests above, and find which motifs are enriched in majority of main populations

In [ ]:
proj <- addMotifAnnotations(ArchRProj = proj, motifSet = "cisbp", name = "Motif", force = TRUE)

## Function

In [ ]:
motif_up_func <- function(markerTest){
    # get motifs up
    motifsUp <- peakAnnoEnrichment(
    seMarker = markerTest,
    ArchRProj = proj,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )
    
    # create df of motifs and rank
    df <- data.frame(TF = rownames(motifsUp), mlog10Padj = assay(motifsUp)[,1])
    df <- df[order(df$mlog10Padj, decreasing = TRUE),]
    df$rank <- seq_len(nrow(df))
    
    # return motif df
    return(df)
    }

## Find Motifs

In [ ]:
sen_motifs <- lapply(sen_age_markers_up, motif_up_func)

In [ ]:
ped_motifs <- lapply(ped_age_markers_up, motif_up_func)

In [ ]:
cmv_pos_motifs <- lapply(cmv_pos_markers_up, motif_up_func)

In [ ]:
cmv_neg_motifs <- lapply(cmv_neg_markers_up, motif_up_func)

## Extract Significant Motifs

In [ ]:
names(sen_motifs) <- celltypes
names(ped_motifs) <- celltypes
names(cmv_pos_motifs) <- celltypes
names(cmv_neg_motifs) <- celltypes
# head(sen_motifs)

In [ ]:
sen_motifs_subset <- list(sen_motifs[['CD8 Naive']],sen_motifs[['CD8 CM']],sen_motifs[['CD8 EM1']],sen_motifs[['CD8 EM2']],sen_motifs[['CD8 TEMRA']],
                          sen_motifs[['CD4 Naive']],sen_motifs[['CD4 CM']],sen_motifs[['CD4 EM1']],sen_motifs[['CD4 EM2']],sen_motifs[['CD4 TEMRA']],
                          sen_motifs[['Treg']])                         
ped_motifs_subset <- list(ped_motifs[['CD8 Naive']],ped_motifs[['CD8 CM']],ped_motifs[['CD8 EM1']],ped_motifs[['CD8 EM2']],ped_motifs[['CD8 TEMRA']],
                          ped_motifs[['CD4 Naive']],ped_motifs[['CD4 CM']],ped_motifs[['CD4 EM1']],ped_motifs[['CD4 EM2']],ped_motifs[['CD4 TEMRA']],
                          ped_motifs[['Treg']])
cmv_pos_motifs_subset <- list(cmv_pos_motifs[['CD8 Naive']],cmv_pos_motifs[['CD8 CM']],cmv_pos_motifs[['CD8 EM1']],cmv_pos_motifs[['CD8 EM2']],cmv_pos_motifs[['CD8 TEMRA']],
                          cmv_pos_motifs[['CD4 Naive']],cmv_pos_motifs[['CD4 CM']],cmv_pos_motifs[['CD4 EM1']],cmv_pos_motifs[['CD4 EM2']],cmv_pos_motifs[['CD4 TEMRA']],
                          cmv_pos_motifs[['Treg']])
cmv_neg_motifs_subset <- list(cmv_neg_motifs[['CD8 Naive']],cmv_neg_motifs[['CD8 CM']],cmv_neg_motifs[['CD8 EM1']],cmv_neg_motifs[['CD8 EM2']],cmv_neg_motifs[['CD8 TEMRA']],
                          cmv_neg_motifs[['CD4 Naive']],cmv_neg_motifs[['CD4 CM']],cmv_neg_motifs[['CD4 EM1']],cmv_neg_motifs[['CD4 EM2']],cmv_neg_motifs[['CD4 TEMRA']],
                          cmv_neg_motifs[['Treg']])

In [ ]:
trim_senior_motifs <- lapply(sen_motifs_subset, function(x){
    trimmed_motifs <- subset(x, mlog10Padj > 5)
    return(trimmed_motifs$TF)
    })
senior_motifs_unlist <- unlist(trim_senior_motifs)
sen_motif_df <- as.data.frame(table(senior_motifs_unlist))
sen_motif_df_top <- subset(sen_motif_df, Freq >= 6)
sen_motif_df_top$senior_motifs_unlist

In [ ]:
trim_ped_motifs <- lapply(ped_motifs_subset, function(x){
    trimmed_motifs <- subset(x, mlog10Padj > 5)
    return(trimmed_motifs$TF)
    })
ped_motifs_unlist <- unlist(trim_ped_motifs)
ped_motif_df <- as.data.frame(table(ped_motifs_unlist))
ped_motif_df_top <- subset(ped_motif_df, Freq >= 6)
ped_motif_df_top$ped_motifs_unlist

In [ ]:
trim_cmv_pos_motifs <- lapply(cmv_pos_motifs_subset, function(x){
    trimmed_motifs <- subset(x, mlog10Padj > 5)
    return(trimmed_motifs$TF)
    })
cmv_pos_motifs_unlist <- unlist(trim_cmv_pos_motifs)
cmv_pos_motif_df <- as.data.frame(table(cmv_pos_motifs_unlist))
# cmv_pos_motif_df
# cmv_pos_motif_df_top <- subset(cmv_pos_motif_df, Freq >= 6)
# cmv_pos_motif_df_top$cmv_pos_motifs_unlist

In [ ]:
trim_cmv_neg_motifs <- lapply(cmv_neg_motifs_subset, function(x){
    trimmed_motifs <- subset(x, mlog10Padj > 5)
    return(trimmed_motifs$TF)
    })
cmv_neg_motifs_unlist <- unlist(trim_cmv_neg_motifs)
cmv_neg_motif_df <- as.data.frame(table(cmv_neg_motifs_unlist))
# cmv_neg_motif_df
# cmv_pos_motif_df_top <- subset(cmv_pos_motif_df, Freq >= 6)
# cmv_pos_motif_df_top$cmv_pos_motifs_unlist

In [ ]:
saveRDS(trim_senior_motifs, file = '../02_Gating_Subsets/trim_senior_motifs_cisbp.rds')
saveRDS(trim_ped_motifs, file = '../02_Gating_Subsets/trim_ped_motif_cisbp.rds')
saveRDS(trim_cmv_pos_motifs, file = '../02_Gating_Subsets/trim_cmv_pos_motifs_cisbp.rds')
saveRDS(trim_cmv_neg_motifs, file = '../02_Gating_Subsets/trim_cmv_neg_motifs_cisbp.rds')

In [ ]:
saveRDS(sen_motifs, file = '../02_Gating_Subsets/sen_motifs_cisbp.rds')
saveRDS(ped_motifs, file = '../02_Gating_Subsets/ped_motifs_cisbp.rds')

## Organize for plotting

In [ ]:
updated_celltype_list <- c('CD8 Naive','CD8 CM','CD8 EM1','CD8 EM2','CD8 TEMRA','CD4 Naive','CD4 CM','CD4 EM1','CD4 EM2','CD4 TEMRA','Treg')

In [ ]:
names(sen_motifs_subset) <- updated_celltype_list
names(ped_motifs_subset) <- updated_celltype_list

In [ ]:
# names(pediatric_motifs) <- paste0('Pediatric_',trim_celltype_list)
names(ped_motifs) <- updated_celltype_list
# head(pediatric_motifs)

In [ ]:
# names(senior_motifs) <- paste0('Senior_',trim_celltype_list)
names(sen_motifs) <- updated_celltype_list

In [ ]:
top_pediatric_motifs <- lapply(ped_motifs_subset, function(x){
    trimmed_motifs <- subset(x, TF %in% ped_motif_df_top$ped_motifs_unlist)
    return(trimmed_motifs)
    })

In [ ]:
top_senior_motifs <- lapply(sen_motifs_subset, function(x){
    trimmed_motifs <- subset(x, TF %in% sen_motif_df_top$senior_motifs_unlist)
    return(trimmed_motifs)
    })

In [ ]:
library(purrr)

In [ ]:
ped_collapsed_tf <- map_df(top_pediatric_motifs, ~as.data.frame(.x), .id="id")

In [ ]:
sen_collapsed_tf <- map_df(top_senior_motifs, ~as.data.frame(.x), .id="id")

In [ ]:
ped_collapsed_tf$mlog10Padj <- (ped_collapsed_tf$mlog10Padj * -1)
head(ped_collapsed_tf)

In [ ]:
tf_bind <- rbind(ped_collapsed_tf,sen_collapsed_tf)
head(tf_bind)

In [ ]:
min(tf_bind$mlog10Padj)
max(tf_bind$mlog10Padj)

In [ ]:
saveRDS(tf_bind, file = '../02_Gating_Subsets/ped_sen_motifs_cisbp.rds')

## Plot

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 10)
ggplot(tf_bind, aes(x = id, y = reorder(TF,mlog10Padj), fill = mlog10Padj)) + geom_tile() + scale_fill_gradientn(colours = c('#2166ac','white','#b2182b'), 
                                                                                             values = scales::rescale(c(-30, -15, 0, 50, 200))) +  
theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
# ggplot(ped_collapsed_tf, aes(x = id, y = TF, fill = mlog10Padj)) + geom_tile() + scale_fill_gradient(low = 'white', high = '#1b9e77') + 
# theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
# options(repr.plot.width = 10, repr.plot.height = 10)
# ggplot(sen_collapsed_tf, aes(x = id, y = TF, fill = mlog10Padj)) + geom_tile() + scale_fill_gradient(low = 'white', high = '#d95f02') + 
# theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

# Session Info

In [ ]:
sessionInfo()